## 基于AB测试的支付宝营销效果分析
案例说明：  
1.数据来源：
本文所用的数据集来源于阿里云天池：https://tianchi.aliyun.com/dataset/dataDetail?dataId=50893&lang=zh-cn

该数据集包含三张表(数据量100万)，分别记录了支付宝两组营销的活动情况：

emb_tb_2.csv: 用户特征数据集   
effect_tb.csv: 广告点击情况数据集  
seed_cand_tb.csv: 用户类型数据集  
本分析报告的主要使用广告点击情况数据，涉及字段如下：

dmp_id：营销策略编号（源数据文档未作说明，这里根据数据情况设定为1：对照组，2：营销策略一，3：营销策略二）  
user_id：支付宝用户ID  
label：用户当天是否点击活动广告（0：未点击，1：点击）

2.分析目的：分析不同营销策略的效果提升情况

## 一、明确AB测试分析流程
（1）分析业务数据，确定实验指标   
（2）建立实验假设   
（3）选取样本  
（4）确定核心指标提升的阈值  
（5）确定样本量  
（6）分配流量、确定实验时长  
（7）采集数据，进行分析。 

### （1）分析业务数据，确定实验指标
该案例由于数据限制，仅包含营销策略选择和用户是否参加活动广告。所以以：是否点击活动广告作为实验指标。

为衡量用户的点击情况，构建点击率指标，即：P=点击用户数/当天登录用户数.   

此外P0表示对照组的点击率,P1表示策略一的点击率,P2表示策略二的点击率

### （2）建立实验假设 
营销策略一：  
H0:营销策略一对用户点击有显著提升，即p1>=p0  
H1:营销策略一对用户点击没有显著提升，即p1<p0

营销策略二：                             
H0:营销策略二对用户点击有显著提升，即p2>=p0  
H1:营销策略二对用户点击没有显著提升，即p2<p0
                          

### （3）选取样本

选取活动范围的登录的用户数

### （4）确定核心指标提升的阈值

假设，根据业务需求提出，需要点击率提升1%

### （5）确定样本量
假定显著性水平为0.05，检验功效为0.8，分布满足正态分布时情况下计算最小样本量：   

同时，假设往期用户的平均点击率为1.5%，想通过这次的活动将点击率提升1%，即将从1.5%提升到2.5%，那么通过如下计算可以计算出最小样本量为3029.

In [1]:
from statsmodels.stats.power import GofChisquarePower
from statsmodels.stats.power import zt_ind_solve_power
from statsmodels.stats.proportion import proportion_effectsize as es

#定显著性水平为0.05，检验功效为0.8，效应量为标准均差计算方式，实验组和对照组数量相等，双边检验。
zt_ind_solve_power(effect_size=es(prop1=0.015, prop2=0.025, method= 'normal'),
                   alpha=0.05, power=0.8, ratio=1.0, alternative="two-sided")


3028.537801878027

### （6）分配流量、确定实验时长
正常的流程，此时应该根据业务的实际情况，分配流量，并且计算实验的时长。   
例如，根据最小样本为3028，那么我们为了保证数据的有效性，选择获取的样本量为4000；    
然后根据业务中采取数据流量的速度，来计算我们实验的时长。即，如果每天只能获取1000的数据，那么我们就需要获取4天的数据。   

### （7）采集数据，进行分析。
假设，由阿里云的提供的数据为我们实际采集的数据，并以此进行测试分析。

## 二、AB测试分析

#### 1.数据处理
主要包含：数据导入与清洗

In [2]:
#（1）数据导入：
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
data = pd.read_csv('effect_tb.csv',header = None)
data.columns = ['dt','user_id','label','dmp_id']

# 日志天数,由于这里缺乏活动的日期范围，这里假设所给数据均为活动时的数据。
data = data.drop(columns='dt')
data.head(3)

,user_id,label,dmp_id
0,1,0,1
1,1000004,0,1
2,1000004,0,2


In [4]:
#（2）数据的基本情况：数据类型
data.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2645958 entries, 0 to 2645957
Data columns (total 3 columns):
 #   Column   Non-Null Count    Dtype
---  ------   --------------    -----
 0   user_id  2645958 non-null  int64
 1   label    2645958 non-null  int64
 2   dmp_id   2645958 non-null  int64
dtypes: int64(3)
memory usage: 60.6 MB


In [5]:
#（2）数据的基本情况：统计量
data.describe()

,user_id,label,dmp_id
count,2.645958e+06,2.645958e+06,2.645958e+06
mean,3.112995e+06,1.456297e-02,1.395761e+00
std,1.828262e+06,1.197952e-01,6.920480e-01
min,1.000000e+00,0.000000e+00,1.000000e+00
25%,1.526772e+06,0.000000e+00,1.000000e+00
50%,3.062184e+06,0.000000e+00,1.000000e+00
75%,4.721132e+06,0.000000e+00,2.000000e+00
max,6.265402e+06,1.000000e+00,3.000000e+00


#### 通过从数据类型和统计量来看：
 （1） 数据均是整数型，不存在字符串   
 （2） 不存在缺失值    
 （3） label,dmp_id不存在异常值  

In [6]:
#（3）判断数据的重复情况：nunique() 可直接统计dataframe中每列的不同值的个数
data.nunique()

user_id    2410683
label            2
dmp_id           3
dtype: int64

In [7]:
#（3）上述结果发现，数据长度与用户数不一致，可能存在重复行，对此进行检验
#duplicated()：对重复值标记
data[data.duplicated(keep=False)].sort_values(by=["user_id"])

,user_id,label,dmp_id
8529,1027,0,1
1485546,1027,0,1
1579415,1471,0,1
127827,1471,0,1
404862,2468,0,1
...,...,...,...
1382121,6264633,0,1
1382245,6264940,0,1
2575140,6264940,0,1
1382306,6265082,0,3


In [8]:
#（3）删除重复数据&检查
data = data.drop_duplicates()

data[data.duplicated(keep = False)]

,user_id,label,dmp_id


#### （3）存在重复行的数据，已经成功剔除

In [9]:
#（4）缺失值判断：
data.isnull().sum()

user_id    0
label      0
dmp_id     0
dtype: int64

#### （4）不存在缺失值

### 2.样本容量检验

根据前面准备的工作中知道我们的最小样本量为3028，而目前获得的样本为200多万，远远超过预期。

### 3.进行假设检验计算

#### （1）核心指标计算

In [10]:
#不同策略下的点击率：
P0=data[data["dmp_id"] == 1]["label"].mean()
P1=data[data["dmp_id"] == 2]["label"].mean()
P2=data[data["dmp_id"] == 3]["label"].mean()

print("对照组点击率：P0={:.4f} ".format(P0))
print("策略一点击率：P1={:.4f} ".format(P1))
print("策略二点击率：P2={:.4f} ".format(P2))

对照组点击率：P0=0.0126 
策略一点击率：P1=0.0153 
策略二点击率：P2=0.0262 


#### 通过计算发现，策略一提升了0.2%，策略二提升了1.3%。根据我们的假定，需要满足对点击率的提升1%个点，所以策略一失效，并对策略二进行检验

#### （2）构建假设检验&确定检验类型

构建假设：  
H0:营销策略二对用户点击没有显著提升，即p2<=p0  
H1:营销策略二对用户点击有显著提升，即p2>p0
                           
检验方向：根据备择假设，确定为单侧反向检验                           

检验类型：
首先样本服从二项分布，独立的双样本，样本容量n>30,总体的均值和方程未知，采用Z检验
检验统计量为：

![](https://pic-stored.oss-cn-beijing.aliyuncs.com/image/QianJianTec1686215218480.png)
其中，Pc为总和点击率（点击率的联合估计）   
    


#### (3)检验统计量的计算

In [11]:
#方法一：公式计算
# 用户数
n_old = len(data[data.dmp_id == 1])  # 对照组
n_new = len(data[data.dmp_id == 3])  # 策略二

# 点击数
c_old = len(data.loc[(data.dmp_id == 1) & (data.label == 1)])
c_new = len(data.loc[(data.dmp_id == 3) & (data.label == 1)])

# 计算点击率
p_old = c_old / n_old
p_new = c_new / n_new

# 总和点击率（点击率的联合估计）
p_c = (c_old + c_new) / (n_old + n_new)

print("总和点击率：{:.4f}".format(p_c))

# 计算检验统计量z
z = (p_old - p_new) / np.sqrt(p_c * (1 - p_c)*(1/n_old + 1/n_new))

print("检验统计量z：{:.4f}".format(z))


# 查α=0.05对应的z分位数,注意备择假设是左尾（单侧）
from scipy.stats import norm
z_alpha = norm.ppf(0.05)
# 若为双侧，则norm.ppf(0.05/2)
print("z_0.05：{:.4f}".format(z_alpha))

总和点击率：0.0145
检验统计量z：-59.4417
z_0.05：-1.6449


#### 根据假设检验，可知拒绝域为{z＜z_0.05} 而统计量z，落入拒绝域，则有理由拒绝原假设，认为在显著性水平为0.05时，策略二点击率的提升在统计上是显著的。

In [12]:
#方法二：python计算

In [14]:
import statsmodels.stats.proportion as sp
# alternative='smaller'代表左尾
z_score, p = sp.proportions_ztest([c_old, c_new], [n_old,n_new], alternative = "smaller")
print("检验统计量z：",z_score,"，p值：", p)

检验统计量z： -59.44168632985996 ，p值： 0.0


#### 这里的p值与拒绝域的检验方式是一致的，p<0.05，拒绝原假设。

#### （4）效应量计算
所谓的效应量主要衡量的是效应的大小，通俗的话即：P值代表的是统计学上的意义，而效应量是能反映实际上的意义。其本质就是把差异和相关性量化的过程。    
常用的效应量有：Cohen'd，中文一般翻译作科恩d值：

$$d=\frac{\left(\text { 样本平均值 }_{1}-\text { 样本平均值 }_{2}\right)}{\text { 标准差 }}$$

这里的标准差，由于是双独立样本的，需要用合并标准差（pooled standard deviations）代替。也就是以合并标准差为单位，计算两个样本平均值之间相差多少。双独立样本的合并标准差可以如下计算：
$$s=\sqrt{\frac{\left(\left(n_{1}-1\right) \times s_{1}^{2}+\left(n_{2}-1\right) \times s_{2}^{2}\right)}{n_{1}+n_{2}-2}}$$

In [19]:
# 合并标准差
std_old = data[data.dmp_id ==1].label.std()
std_new = data[data.dmp_id ==3].label.std()
s = np.sqrt(((n_old - 1)* std_old**2 + (n_new - 1)* std_new**2 ) / (n_old + n_new - 2))
# 效应量Cohen's d
d = (p_old - p_new) / s
print("对照组样本标准差：{:.4f}".format(std_old))
print("策略二样本标准差：{:.4f}".format(std_new))
print("Cohen\'s d为：：{:.4f}".format(d))

对照组样本标准差：0.1113
策略二样本标准差：0.1597
Cohen's d为：：-0.1142


#### 一般上baiCohen's d取值0.2-0.5为小效应，0.5-0.8中等效应，0.8以上为大效应。计算出为-0.0114，可见效应量较小。

# 三、输出分析结论

#### 1.假设检验报告  

（1）假定变量：a=0.05;   
（2）计算结果：   
    对照组：点击率P0=0.0126，标准差为：0.11   
    策略二：点击率P2=0.0262，标准差为：0.16    
    统计量：Z=-59.44 ，p=0 ； 单侧左尾检验，拒绝原假设   
    效应量：Cohen's d=-0.11  数值较小    

#### 2.结论与建议
通过A/B测试的方法，在两种营销策略中，仅只有策略二满足提升1%的要求。同时，策略二对广告点击率有显著的提升效果，从0.012提升到0.026提升一倍多，因此相比营销策略一，应该选择营销策略二进行推广